In [3]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from statistics import mean
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
import shap
from scipy.stats import ranksums

In [1]:
MR = [5, 40, 100]
N_genes = 100  # total no. of genes
N_TFs = N_genes

n_estimators=[500, 1000, 2000]  # number of trees in the forest
criterion='squared_error'  # variance reduction equivalent
max_features = [N_TFs, 'sqrt'] # max no. of features to use in each split 
random_state = 42  # for reproducibility

In [3]:
#rf_results = pd.DataFrame(columns=['MR', 'FI', 'N_EST', 'MAX_FEATURES','BOOTSTRAPPING', 'AUPRC', 'AUROC', 'MEAN_AUROC','STD_AUROC','MEAN_AUPRC','STD_AUPRC', 'p-value', 'BOOSTING'])
#rf_results.to_csv("results/grn_rf_gb_results.csv", index=False)     


In [11]:
for mr in MR:
    for n_est in n_estimators:
        for m_f in max_features:
                
                rf_results = pd.read_csv("results/grn_rf_gb_results.csv", header=0)
                
                data_file = ("data/{}_mr_50_cond/simulated_noNoise.txt").format(mr)
                grn_file = ("data/{}_mr_50_cond/bipartite_GRN.csv").format(mr)

                data = pd.read_csv(data_file, sep="\t", header=0)
                grn_df = pd.read_csv(grn_file, sep = ",", header = None, names=['TF_ID', 'G_ID'])
                grn_df['class'] = 1

                # Normalize Expression data to unit-variance
                data_n = StandardScaler(with_mean=False).fit_transform(data.to_numpy())

                # Initialize matrices
                W = np.zeros(shape=(N_genes,N_TFs))
                W_shap = np.zeros(shape=(N_genes,N_TFs))
                Fscores = np.zeros(shape=(N_genes,))

                for j in np.arange(0,N_genes):
                    # read TF and gene expression data X and Gj
                    X, Gj= data_n[:,:N_TFs], data_n[:,N_genes+j]

                    # fit an RF model to predict gene expression from TF
                    M_rf_gb = GradientBoostingRegressor(criterion=criterion, n_estimators=n_est, loss='squared_error', max_features=m_f, random_state=random_state).fit(X,Gj)
                    # train score
                    Fscores[j] = M_rf_gb.score(X,Gj)

                    # Get the weights for all edges connecting TFs to gene j
                    W[j,:] = M_rf_gb.feature_importances_

                    # look at feature importance based on SHAP values
                    explainer = shap.TreeExplainer(M_rf_gb)
                    shap_values = explainer(X)
                    W_shap[j,:] = np.mean(np.abs(shap_values.values), axis=0)
                    
                W_df = pd.DataFrame(np.abs(W))

                grn_pred = pd.melt(W_df.reset_index(), id_vars = 'index', var_name='TF_ID', value_name='W_pred').rename(columns={'index': 'G_ID'})

                grn_pred['G_ID'] = grn_pred['G_ID'].astype(np.int64) + 100
                grn_pred['TF_ID'] = grn_pred['TF_ID'].astype(np.int64)

                grn_eval = pd.merge(grn_pred,grn_df, on=['G_ID', 'TF_ID'], how='left')
                grn_eval['class'] = grn_eval['class'].fillna(int(0))

                grn_eval.to_csv("results/{}_mr_50_cond/grn_eval_rf_vr_{}_{}_gb_{}.csv".format(mr,n_est,m_f,'squared_error'), index=False)

                precision, recall, thresholds_prc = precision_recall_curve(grn_eval['class'], grn_eval['W_pred'])
                fpr, tpr, thresholds_roc = roc_curve(grn_eval['class'], grn_eval['W_pred'])
                auprc = auc(recall, precision)
                auroc = auc(fpr,tpr)

                roc_gene = [] 
                ap_gene = []
                for i in range(100):
                    grn_eval_gene = grn_eval.iloc[i::N_TFs,:]
                    roc_gene.append(metrics.roc_auc_score(grn_eval_gene['class'], grn_eval_gene['W_pred']))
                    ap_gene.append(metrics.average_precision_score(grn_eval_gene['class'], grn_eval_gene['W_pred']))

                mean_auroc = mean(roc_gene)
                sd_auroc = np.std(roc_gene)
                
                mean_auprc = mean(ap_gene)
                sd_auprc = np.std(ap_gene)

                ranksums_pvalue = ranksums(grn_eval[grn_eval['class']==1]['W_pred'], grn_eval[grn_eval['class']==0]['W_pred'], alternative='greater')

                prc = pd.DataFrame({'precision': precision, 'recall': recall}, columns=['precision', 'recall'])
                roc = pd.DataFrame({'fpr': fpr, 'tpr': tpr}, columns=['fpr', 'tpr'])
                prc.to_csv("results/{}_mr_50_cond/grn_prc_rf_vr_{}_{}_gb_{}.csv".format(mr,n_est,m_f,'squared_error'), index=False)
                roc.to_csv("results/{}_mr_50_cond/grn_roc_rf_vr_{}_{}_gb_{}.csv".format(mr,n_est,m_f,'squared_error'), index=False)
                
                temp1 = pd.DataFrame([[mr, "VR", n_est, m_f, False, auprc, auroc, mean_auroc, sd_auroc, mean_auprc, sd_auprc, ranksums_pvalue[1], "GradientBoosting" + 'squared_error']], \
                        columns=['MR', 'FI', 'N_EST', 'MAX_FEATURES','BOOTSTRAPPING', 'AUPRC', 'AUROC', 'MEAN_AUROC','STD_AUROC','MEAN_AUPRC','STD_AUPRC', 'p-value', 'BOOSTING'])
         
                W_shap_df = pd.DataFrame(np.abs(W_shap))

                grn_pred_shap = pd.melt(W_shap_df.reset_index(), id_vars = 'index', var_name='TF_ID', value_name='W_pred').rename(columns={'index': 'G_ID'})

                grn_pred_shap['G_ID'] = grn_pred_shap['G_ID'].astype(np.int64) + 100
                grn_pred_shap['TF_ID'] = grn_pred_shap['TF_ID'].astype(np.int64)

                grn_eval_shap = pd.merge(grn_pred_shap,grn_df, on=['G_ID', 'TF_ID'], how='left')
                grn_eval_shap['class'] = grn_eval_shap['class'].fillna(int(0)) == 1.0

                grn_eval_shap.to_csv("results/{}_mr_50_cond/grn_eval_rf_shap_{}_{}_gb_{}.csv".format(mr,n_est,m_f,'squared_error'), index=False)

                precision, recall, thresholds_prc = precision_recall_curve(grn_eval_shap['class'], grn_eval_shap['W_pred'])
                fpr, tpr, thresholds_roc = roc_curve(grn_eval_shap['class'], grn_eval_shap['W_pred'])
                auprc = auc(recall, precision)
                auroc = auc(fpr,tpr)

                roc_gene = [] 
                ap_gene = []
                for i in range(100):
                    grn_eval_gene = grn_eval_shap.iloc[i::N_TFs,:]
                    roc_gene.append(metrics.roc_auc_score(grn_eval_gene['class'], grn_eval_gene['W_pred']))
                    ap_gene.append(metrics.average_precision_score(grn_eval_gene['class'], grn_eval_gene['W_pred']))

                mean_auroc = mean(roc_gene)
                sd_auroc = np.std(roc_gene)
                
                mean_auprc = mean(ap_gene)
                sd_auprc = np.std(ap_gene)

                mean_auroc = mean(roc_gene)
                ranksums_pvalue = ranksums(grn_eval_shap[grn_eval_shap['class']==1]['W_pred'], grn_eval_shap[grn_eval_shap['class']==0]['W_pred'], alternative='greater')

                prc = pd.DataFrame({'precision': precision, 'recall': recall}, columns=['precision', 'recall'])
                roc = pd.DataFrame({'fpr': fpr, 'tpr': tpr}, columns=['fpr', 'tpr'])
                prc.to_csv("results/{}_mr_50_cond/grn_prc_rf_shap_{}_{}_gb_{}.csv".format(mr,n_est,m_f,'squared_error'), index=False)
                roc.to_csv("results/{}_mr_50_cond/grn_roc_rf_shap_{}_{}_gb_{}.csv".format(mr,n_est,m_f,'squared_error'), index=False)
                
                temp2 = pd.DataFrame([[mr, "SHAP", n_est, m_f, False, auprc, auroc, mean_auroc,sd_auroc, mean_auprc, sd_auprc, ranksums_pvalue[1], "GradientBoosting" + 'squared_error']], \
                        columns=['MR', 'FI', 'N_EST', 'MAX_FEATURES','BOOTSTRAPPING', 'AUPRC', 'AUROC', 'MEAN_AUROC','STD_AUROC','MEAN_AUPRC','STD_AUPRC', 'p-value', 'BOOSTING'])
                
                df_list = [rf_results, temp1, temp2]
                df_out = pd.concat([df for df in df_list if not df.empty])                
                df_out.to_csv("results/grn_rf_gb_results.csv", index=False)